In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
def myPrint(x):
    print(x.head(15))
    print(x.info())
def myPrint2(x):
    print(x)

In [4]:
test_b_wa_df = pd.read_csv('../data/Test-B/test_b_wa_df.csv', header = 0)

In [5]:
myPrint(test_b_wa_df)

     uid  wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   7000  139上网导航          1          75       60         76        0    10
1   7000  139上网导航          2        1001     2058       1131        0     7
2   7000  139上网导航          3        1020     1721        615        0    20
3   7000  139上网导航          3         370     2782      12065        0    23
4   7000  139上网导航          5         711     1769        739        0     2
5   7000  139上网导航          5        8934     5933      22796        0     1
6   7000  139上网导航          6        9228     6819      15712        0    26
7   7000  139上网导航          7        8285     9316      18325        0     3
8   7000  139上网导航          8       16637    15256      17954        0    11
9   7000  139上网导航          8        7858     8869       3364        0    17
10  7000  139上网导航         10       12688    33456     629756        0    18
11  7000  139上网导航         14       19113    13871      27719        0    21
12  7000  13

# Feature 21

In [6]:
group = test_b_wa_df.groupby('uid')['visit_cnt']
print(group.unique())

uid
7000    [1, 2, 3, 5, 6, 7, 8, 10, 14, 15, 18, 20, 4, 9...
7001    [1, 6, 17, 29, 31, 34, 38, 75, 79, 125, 128, 3...
7002    [1, 4, 6, 3, 2, 83, 8, 11, 15, 116, 16, 94, 12...
7003    [2, 29, 1, 28, 102, 290, 3, 4, 6, 11, 14, 40, ...
7004                                               [1, 2]
7005                        [2, 16, 12, 17, 1, 4, 10, 13]
7006    [3, 2, 4, 10, 6, 14, 1, 29, 9, 5, 8, 20, 17, 1...
7007    [1, 157, 199, 414, 4, 2, 3, 6, 7, 5, 11, 14, 2...
7008    [1, 4, 2, 8, 9, 19, 3, 10, 49, 5, 7, 209, 338,...
7009    [1, 3, 14, 22, 32, 94, 2, 4, 5, 7, 6, 8, 9, 10...
7010    [2, 10, 1, 14, 3, 114, 126, 347, 4, 8, 13, 22,...
7011    [3, 1, 2, 7, 12, 4, 5, 15, 9, 19, 8, 11, 13, 5...
7012    [22, 48, 1, 2, 3, 4, 5, 8, 7, 16, 10, 27, 29, ...
7013    [1, 3, 5, 2, 7, 50, 4, 6, 12, 38, 20, 23, 72, ...
7014    [2, 3, 1, 13, 21, 97, 138, 200, 4, 5, 8, 12, 8...
7015    [1, 4, 8, 12, 28, 10, 18, 71, 5, 2, 3, 15, 6, ...
7016    [1, 2, 4, 3, 6, 8, 19, 693, 758, 1284, 5, 10, ...
7017      

In [7]:
visit_cnt = group.agg(['std', 'var', 'max','min','median','mean','sum', 'count', 'mad']).add_prefix('wa_visit_cnt_')

In [8]:
print(visit_cnt)

      wa_visit_cnt_std  wa_visit_cnt_var  wa_visit_cnt_max  wa_visit_cnt_min  \
uid                                                                            
7000        140.846632      19837.773710              2708                 1   
7001         55.714521       3104.107905               346                 1   
7002         37.565650       1411.178033               579                 1   
7003         31.659046       1002.295194               290                 1   
7004          0.351866          0.123810                 2                 1   
7005          6.139406         37.692308                17                 1   
7006        129.148286      16679.279717              1158                 1   
7007         96.142591       9243.397819               695                 1   
7008         72.783322       5297.412014               728                 1   
7009        156.868892      24607.849164              1574                 1   
7010        107.141357      11479.270313

# Feature 22

In [9]:
visit_dura = test_b_wa_df.groupby(['uid'])['visit_dura'].agg(['std', 'var', 'max','min','median','mean','sum', 'count', 'mad']).add_prefix('wa_visit_dura_')

In [10]:
print(visit_dura)

      wa_visit_dura_std  wa_visit_dura_var  wa_visit_dura_max  \
uid                                                             
7000      171325.765601       2.935252e+10            3171398   
7001       78365.237426       6.141110e+09             563774   
7002       60325.848017       3.639208e+09             766186   
7003       38996.743593       1.520746e+09             282609   
7004        3321.793286       1.103431e+07               9144   
7005       16541.679576       2.736272e+08              54543   
7006      136911.725120       1.874482e+10            1154015   
7007      117815.362123       1.388046e+10             871072   
7008       80045.382785       6.407263e+09             809644   
7009      244172.829596       5.962037e+10            2456950   
7010      189301.875571       3.583520e+10            3364786   
7011      624270.064878       3.897131e+11           14093288   
7012       52795.668376       2.787383e+09             374358   
7013      377006.980386  

# Feature 23

In [11]:
up_flow = test_b_wa_df.groupby(['uid'])['up_flow'].agg(['std', 'var', 'max','min','median','mean','sum', 'count', 'mad']).add_prefix('wa_up_flow_')

In [12]:
print(up_flow)

      wa_up_flow_std  wa_up_flow_var  wa_up_flow_max  wa_up_flow_min  \
uid                                                                    
7000    1.122567e+06    1.260156e+12        25711352               0   
7001    2.149085e+05    4.618566e+10         1979881               0   
7002    6.716187e+05    4.510717e+11         9596903               0   
7003    2.760799e+05    7.622009e+10         7151849               0   
7004    6.127321e+02    3.754406e+05            1481              65   
7005    2.929589e+04    8.582490e+08           98085             185   
7006    4.962358e+05    2.462500e+11         6936774               0   
7007    1.154815e+06    1.333599e+12        30426643               0   
7008    1.320048e+06    1.742527e+12        37508385               0   
7009    9.505502e+05    9.035457e+11        23942422               0   
7010    7.485356e+05    5.603055e+11        13880594               0   
7011    6.739888e+06    4.542609e+13       205950733            

# Feature 24

In [13]:
down_flow = test_b_wa_df.groupby(['uid'])['down_flow'].agg(['std', 'var', 'max','min','median','mean','sum', 'count', 'mad']).add_prefix('wa_down_flow_')

In [14]:
print(down_flow)

      wa_down_flow_std  wa_down_flow_var  wa_down_flow_max  wa_down_flow_min  \
uid                                                                            
7000      2.895230e+07      8.382357e+14        1021348425                 0   
7001      6.767106e+06      4.579372e+13          79887347                 0   
7002      1.602130e+07      2.566821e+14         261645404                 0   
7003      4.979388e+06      2.479431e+13          86040871                 0   
7004      7.252779e+02      5.260280e+05              2264                52   
7005      8.920398e+04      7.957350e+09            302756               736   
7006      2.450475e+07      6.004826e+14         411226993                 0   
7007      1.297581e+07      1.683716e+14         220884825                 0   
7008      7.416592e+06      5.500583e+13         126921802                 0   
7009      4.953227e+06      2.453446e+13          59678917                 0   
7010      2.810752e+07      7.900327e+14

# Feature 25

In [15]:
group = test_b_wa_df.groupby('uid')['wa_name']

In [16]:
agg = group.aggregate({'wa_name': lambda x: x.nunique()})

d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [17]:
agg.rename(columns=lambda x:x.replace('wa_name','wa_name_count'), inplace=True)

In [18]:
print(agg)

      wa_name_count
uid                
7000            220
7001             45
7002             98
7003            145
7004              2
7005             11
7006            100
7007            135
7008            246
7009            258
7010            183
7011            409
7012             28
7013            275
7014            218
7015            276
7016            286
7017             26
7018            229
7019            124
7020            196
7021             29
7022            324
7023             65
7024            329
7025            517
7026            195
7027            195
7028            260
7029            124
...             ...
9970            279
9971            215
9972            258
9973            245
9974            361
9975            133
9976            132
9977            247
9978            143
9979            287
9980            491
9981            216
9982             36
9983             91
9984            123
9985            306
9986             68


In [19]:
wa_name = agg

# Feature 26

In [20]:
wa_type = test_b_wa_df.groupby(['uid','wa_type'])['uid'].count().unstack().add_prefix('wa_type_').fillna(0)

In [21]:
wa_type.columns.names = ['']

In [22]:
print(wa_type)

      wa_type_0  wa_type_1
uid                       
7000     1483.0      334.0
7001      161.0       25.0
7002      316.0       80.0
7003      943.0      172.0
7004       15.0        0.0
7005       12.0        2.0
7006      257.0       56.0
7007      639.0      111.0
7008     1260.0      193.0
7009     1042.0      291.0
7010      861.0      208.0
7011     2140.0      358.0
7012      257.0       43.0
7013     1267.0      332.0
7014     1004.0      241.0
7015     1102.0      218.0
7016     1962.0      504.0
7017       24.0        5.0
7018      807.0      178.0
7019      868.0      129.0
7020     1042.0      275.0
7021       76.0       13.0
7022     1729.0      201.0
7023      517.0      106.0
7024     2043.0      457.0
7025     2947.0      516.0
7026      858.0      168.0
7027     1273.0      201.0
7028     1577.0      284.0
7029      737.0      272.0
...         ...        ...
9970     1365.0      318.0
9971     1203.0      190.0
9972     1518.0      278.0
9973     1310.0      256.0
9

# Feature 27

In [23]:
date = test_b_wa_df.groupby(['uid','date'])['uid'].count().unstack().add_prefix('date_').fillna(0)

In [24]:
date.columns.names = ['']

In [25]:
print(date)

      date_1  date_2  date_3  date_4  date_5  date_6  date_7  date_8  date_9  \
uid                                                                            
7000    54.0    56.0    66.0    39.0    67.0    42.0    54.0    32.0     5.0   
7001     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7002     4.0    32.0    19.0     1.0     1.0     8.0    15.0     4.0     9.0   
7003     2.0    18.0    40.0    21.0    14.0    23.0    29.0    43.0    15.0   
7004     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7005     0.0     0.0     0.0    14.0     0.0     0.0     0.0     0.0     0.0   
7006     0.0     0.0     0.0     0.0    17.0     0.0     0.0     0.0     0.0   
7007     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7008    58.0    12.0    13.0    32.0    63.0    58.0    67.0    41.0    12.0   
7009    36.0    33.0    17.0    27.0    32.0    27.0    31.0    45.0    26.0   
7010    32.0    77.0    69.0    64.0    

# Merge Feature 21 to 27

In [26]:
agg = visit_cnt
agg = agg.join(visit_dura)
agg = agg.join(up_flow)
agg = agg.join(down_flow)
agg = agg.join(wa_name)
agg = agg.join(wa_type)
agg = agg.join(date)

In [27]:
print(agg)

      wa_visit_cnt_std  wa_visit_cnt_var  wa_visit_cnt_max  wa_visit_cnt_min  \
uid                                                                            
7000        140.846632      19837.773710              2708                 1   
7001         55.714521       3104.107905               346                 1   
7002         37.565650       1411.178033               579                 1   
7003         31.659046       1002.295194               290                 1   
7004          0.351866          0.123810                 2                 1   
7005          6.139406         37.692308                17                 1   
7006        129.148286      16679.279717              1158                 1   
7007         96.142591       9243.397819               695                 1   
7008         72.783322       5297.412014               728                 1   
7009        156.868892      24607.849164              1574                 1   
7010        107.141357      11479.270313

In [28]:
print(agg.columns)

Index([u'wa_visit_cnt_std', u'wa_visit_cnt_var', u'wa_visit_cnt_max',
       u'wa_visit_cnt_min', u'wa_visit_cnt_median', u'wa_visit_cnt_mean',
       u'wa_visit_cnt_sum', u'wa_visit_cnt_count', u'wa_visit_cnt_mad',
       u'wa_visit_dura_std', u'wa_visit_dura_var', u'wa_visit_dura_max',
       u'wa_visit_dura_min', u'wa_visit_dura_median', u'wa_visit_dura_mean',
       u'wa_visit_dura_sum', u'wa_visit_dura_count', u'wa_visit_dura_mad',
       u'wa_up_flow_std', u'wa_up_flow_var', u'wa_up_flow_max',
       u'wa_up_flow_min', u'wa_up_flow_median', u'wa_up_flow_mean',
       u'wa_up_flow_sum', u'wa_up_flow_count', u'wa_up_flow_mad',
       u'wa_down_flow_std', u'wa_down_flow_var', u'wa_down_flow_max',
       u'wa_down_flow_min', u'wa_down_flow_median', u'wa_down_flow_mean',
       u'wa_down_flow_sum', u'wa_down_flow_count', u'wa_down_flow_mad',
       u'wa_name_count', u'wa_type_0', u'wa_type_1', u'date_1', u'date_2',
       u'date_3', u'date_4', u'date_5', u'date_6', u'date_7', u'date_8

In [51]:
dict = agg.to_dict()
agg2 = pd.DataFrame(index = range(7000, 10000), data = dict)

In [52]:
agg2 = agg2.fillna(0)
agg2 = agg2.astype('int64')

In [54]:
agg2.to_csv('../data/Test-B/test_b_feature_21to27.csv', index=True, header=None)